Task 2
Scrape job listings from the website https://realpython.github.io/fake-jobs and store the data into an SQLite database.

Scraping Requirements:

Extract the following details for each job listing:
Job Title
Company Name
Location
Job Description
Application Link
Data Storage:

Store the scraped data into an SQLite database in a table named jobs.
Incremental Load:

Ensure that your script performs incremental loading:
Scrape the webpage and add only new job listings to the database.
Avoid duplicating entries. Use Job Title, Company Name, and Location as unique identifiers for comparison.
Update Tracking:

Add functionality to detect if an existing job listing has been updated (e.g., description or application link changes) and update the database record accordingly.
Filtering and Exporting:

Allow filtering job listings by location or company name.
Write a function to export filtered results into a CSV file.

In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import csv

# Constants
URL = "https://realpython.github.io/fake-jobs"
DB_NAME = "jobs.db"

# Function to create the database and table
def initialize_database():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS jobs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            job_title TEXT,
            company_name TEXT,
            location TEXT,
            description TEXT,
            application_link TEXT,
            UNIQUE(job_title, company_name, location)
        )
    ''')
    conn.commit()
    conn.close()

# Function to scrape job listings
def scrape_jobs():
    response = requests.get(URL)
    soup = BeautifulSoup(response.content, "html.parser")

    jobs = []
    for job_element in soup.find_all("div", class_="card-content"):
        job_title = job_element.find("h2", class_="title is-5").text.strip()
        company_name = job_element.find("h3", class_="subtitle is-6 company").text.strip()
        location = job_element.find("p", class_="location").text.strip()
        description = job_element.find("div", class_="content").text.strip()
        application_link = job_element.find("a", text="Apply")['href']

        jobs.append({
            "job_title": job_title,
            "company_name": company_name,
            "location": location,
            "description": description,
            "application_link": application_link,
        })
    return jobs

# Function to store jobs into the database
def store_jobs(jobs):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()

    for job in jobs:
        cursor.execute('''
            INSERT OR IGNORE INTO jobs (job_title, company_name, location, description, application_link)
            VALUES (?, ?, ?, ?, ?)
        ''', (job['job_title'], job['company_name'], job['location'], job['description'], job['application_link']))

        # Check if an existing job has been updated
        cursor.execute('''
            UPDATE jobs
            SET description = ?, application_link = ?
            WHERE job_title = ? AND company_name = ? AND location = ?
              AND (description != ? OR application_link != ?)
        ''', (job['description'], job['application_link'], job['job_title'], job['company_name'], job['location'], job['description'], job['application_link']))

    conn.commit()
    conn.close()

# Function to filter jobs by location or company name
def filter_jobs(location=None, company_name=None):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    query = "SELECT * FROM jobs WHERE 1=1"
    params = []

    if location:
        query += " AND location = ?"
        params.append(location)
    if company_name:
        query += " AND company_name = ?"
        params.append(company_name)

    cursor.execute(query, params)
    results = cursor.fetchall()
    conn.close()
    return results

# Function to export filtered jobs to a CSV file
def export_to_csv(jobs, filename):
    with open(filename, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["Job Title", "Company Name", "Location", "Description", "Application Link"])
        for job in jobs:
            writer.writerow(job)

# Main execution flow
if __name__ == "__main__":
    # Initialize database
    initialize_database()

    # Scrape jobs from the webpage
    jobs = scrape_jobs()

    # Store jobs into the database
    store_jobs(jobs)

    # Example usage: Filter jobs by location or company name
    location_filter = "Remote"
    company_filter = "Company XYZ"
    filtered_jobs = filter_jobs(location=location_filter, company_name=company_filter)

    # Export filtered jobs to a CSV file
    export_to_csv(filtered_jobs, "filtered_jobs.csv")

    print("Job scraping and processing complete.")
